# TimeSeriesSplit – Lab & Lecture (Parameter-by-Parameter Analysis)
**Author:** Auto-generated for teaching

**Prereqs:** NumPy, basic scikit-learn, understanding of temporal order

## 0) Learning Goals / วัตถุประสงค์การเรียนรู้
- Understand **why** classic K‑Fold is inappropriate for time series.
- Master `TimeSeriesSplit` parameters: `n_splits`, `test_size`, `gap`, `max_train_size`.
- Interpret generated **train/test indices** and connect them to calendar time.
- Build small forecasting experiments with proper CV (walk‑forward validation).
- Diagnose **leakage** and choose reasonable parameter values.

## 1) Quick Start Example (from your snippet)

In [1]:
import numpy as np
from sklearn.model_selection import TimeSeriesSplit

X = np.array([[1, 2], [3, 4], [1, 2], [3, 4], [1, 2], [3, 4]])
y = np.array([1, 2, 3, 4, 5, 6])

# With only 6 samples, use a smaller n_splits
tscv = TimeSeriesSplit(n_splits=3)
for i, (train_index, test_index) in enumerate(tscv.split(X)):
    print(f"Split {i+1} -> TRAIN: {train_index}, TEST: {test_index}")
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

Split 1 -> TRAIN: [0 1 2], TEST: [3]
Split 2 -> TRAIN: [0 1 2 3], TEST: [4]
Split 3 -> TRAIN: [0 1 2 3 4], TEST: [5]


## 2) Why not classic K‑Fold? / ทำไม K‑Fold แบบทั่วไปใช้กับ Time Series ไม่เหมาะ
- K‑Fold randomly partitions data into folds → **breaks temporal order**.
- Past must not peek at future → **data leakage** if future samples end up in train.
- `TimeSeriesSplit` ensures all test indices occur **after** the training indices.

In [2]:
from sklearn.model_selection import KFold
Xdemo = np.arange(12).reshape(-1, 1)
print("KFold (BAD for time series):")
for tr, te in KFold(n_splits=3, shuffle=True, random_state=42).split(Xdemo):
    print("train max:", tr.max(), "| test min:", te.min())  # test can be before train!

print("\nTimeSeriesSplit (GOOD):")
for tr, te in TimeSeriesSplit(n_splits=3).split(Xdemo):
    print("train max:", tr.max(), "| test min:", te.min())  # test >= train

KFold (BAD for time series):
train max: 11 | test min: 0
train max: 10 | test min: 1
train max: 11 | test min: 3

TimeSeriesSplit (GOOD):
train max: 2 | test min: 3
train max: 5 | test min: 6
train max: 8 | test min: 9


## 3) The Four Parameters (Deep Dive)
Helper function to visualize splits:

In [3]:
import numpy as np
from sklearn.model_selection import TimeSeriesSplit

def show_splits(n_samples=20, **kw):
    X = np.arange(n_samples).reshape(-1, 1)
    tscv = TimeSeriesSplit(**kw)
    for i, (tr, te) in enumerate(tscv.split(X), 1):
        tr_preview = f"{tr[:3]}...{tr[-3:]}" if len(tr) > 6 else str(tr)
        print(f"Split {i:>2} | train={tr_preview} (len={len(tr)}) | test={te} (len={len(te)})")

        

### 3.1 `n_splits`
- **Definition**: Number of train→test splits (walk‑forward steps).
- **Effect**: More splits → more evaluations but smaller early train windows.
- **Constraint**: Needs enough samples to form at least one test fold.

In [7]:
print('n_splits exploration (n_samples=20)')
show_splits(n_samples=30, n_splits=3)
print('\n---')
show_splits(n_samples=30, n_splits=5)
print('\n---')
show_splits(n_samples=30, n_splits=10)

n_splits exploration (n_samples=20)
Split  1 | train=[0 1 2]...[6 7 8] (len=9) | test=[ 9 10 11 12 13 14 15] (len=7)
Split  2 | train=[0 1 2]...[13 14 15] (len=16) | test=[16 17 18 19 20 21 22] (len=7)
Split  3 | train=[0 1 2]...[20 21 22] (len=23) | test=[23 24 25 26 27 28 29] (len=7)

---
Split  1 | train=[0 1 2 3 4] (len=5) | test=[5 6 7 8 9] (len=5)
Split  2 | train=[0 1 2]...[7 8 9] (len=10) | test=[10 11 12 13 14] (len=5)
Split  3 | train=[0 1 2]...[12 13 14] (len=15) | test=[15 16 17 18 19] (len=5)
Split  4 | train=[0 1 2]...[17 18 19] (len=20) | test=[20 21 22 23 24] (len=5)
Split  5 | train=[0 1 2]...[22 23 24] (len=25) | test=[25 26 27 28 29] (len=5)

---
Split  1 | train=[0 1 2]...[7 8 9] (len=10) | test=[10 11] (len=2)
Split  2 | train=[0 1 2]...[ 9 10 11] (len=12) | test=[12 13] (len=2)
Split  3 | train=[0 1 2]...[11 12 13] (len=14) | test=[14 15] (len=2)
Split  4 | train=[0 1 2]...[13 14 15] (len=16) | test=[16 17] (len=2)
Split  5 | train=[0 1 2]...[15 16 17] (len=18) | 

### 3.2 `test_size`
- **Definition**: Size of each test fold.
- **Use when** you want a fixed horizon (e.g., forecast next 7 days each split).

In [7]:
print('test_size exploration (n_samples=20, n_splits=4)')
show_splits(n_samples=20, n_splits=4, test_size=3)
print('\n---')
show_splits(n_samples=20, n_splits=4, test_size=1)

test_size exploration (n_samples=20, n_splits=4)
Split  1 | train=[0 1 2]...[5 6 7] (len=8) | test=[ 8  9 10] (len=3)
Split  2 | train=[0 1 2]...[ 8  9 10] (len=11) | test=[11 12 13] (len=3)
Split  3 | train=[0 1 2]...[11 12 13] (len=14) | test=[14 15 16] (len=3)
Split  4 | train=[0 1 2]...[14 15 16] (len=17) | test=[17 18 19] (len=3)

---
Split  1 | train=[0 1 2]...[13 14 15] (len=16) | test=[16] (len=1)
Split  2 | train=[0 1 2]...[14 15 16] (len=17) | test=[17] (len=1)
Split  3 | train=[0 1 2]...[15 16 17] (len=18) | test=[18] (len=1)
Split  4 | train=[0 1 2]...[16 17 18] (len=19) | test=[19] (len=1)


### 3.3 `gap`
- **Definition**: Number of samples skipped between train end and test start.
- **Why**: Prevents leakage when features use trailing windows (lags/rolling).

In [8]:
print('gap exploration (n_samples=18, n_splits=4, test_size=2, gap=2)')
show_splits(n_samples=18, n_splits=4, test_size=2, gap=2)

gap exploration (n_samples=18, n_splits=4, test_size=2, gap=2)
Split  1 | train=[0 1 2]...[5 6 7] (len=8) | test=[10 11] (len=2)
Split  2 | train=[0 1 2]...[7 8 9] (len=10) | test=[12 13] (len=2)
Split  3 | train=[0 1 2]...[ 9 10 11] (len=12) | test=[14 15] (len=2)
Split  4 | train=[0 1 2]...[11 12 13] (len=14) | test=[16 17] (len=2)


### 3.4 `max_train_size`
- **Definition**: Cap the training window length. `None` = expanding; integer = sliding window.

In [9]:
print('Expanding window (default)')
show_splits(n_samples=18, n_splits=4, test_size=2)
print('\nSliding window of last 8 samples only')
show_splits(n_samples=18, n_splits=4, test_size=2, max_train_size=8)

Expanding window (default)
Split  1 | train=[0 1 2]...[7 8 9] (len=10) | test=[10 11] (len=2)
Split  2 | train=[0 1 2]...[ 9 10 11] (len=12) | test=[12 13] (len=2)
Split  3 | train=[0 1 2]...[11 12 13] (len=14) | test=[14 15] (len=2)
Split  4 | train=[0 1 2]...[13 14 15] (len=16) | test=[16 17] (len=2)

Sliding window of last 8 samples only
Split  1 | train=[2 3 4]...[7 8 9] (len=8) | test=[10 11] (len=2)
Split  2 | train=[4 5 6]...[ 9 10 11] (len=8) | test=[12 13] (len=2)
Split  3 | train=[6 7 8]...[11 12 13] (len=8) | test=[14 15] (len=2)
Split  4 | train=[ 8  9 10]...[13 14 15] (len=8) | test=[16 17] (len=2)


## 4) Putting It Together (common recipes)

In [10]:
# Recipe A: Next‑k prediction, expanding window (horizon=3)
print('Recipe A: expanding window, horizon=3')
show_splits(n_samples=24, n_splits=5, test_size=3)

# Recipe B: Next‑k prediction, sliding window with leakage guard
print('\nRecipe B: sliding window, horizon=2, gap=3, max_train_size=12')
show_splits(n_samples=30, n_splits=6, test_size=2, max_train_size=12, gap=3)

# Recipe C: Many short checks (micro‑tests)
print('\nRecipe C: many micro‑tests (test_size=1)')
show_splits(n_samples=15, n_splits=7, test_size=1)

Recipe A: expanding window, horizon=3
Split  1 | train=[0 1 2]...[6 7 8] (len=9) | test=[ 9 10 11] (len=3)
Split  2 | train=[0 1 2]...[ 9 10 11] (len=12) | test=[12 13 14] (len=3)
Split  3 | train=[0 1 2]...[12 13 14] (len=15) | test=[15 16 17] (len=3)
Split  4 | train=[0 1 2]...[15 16 17] (len=18) | test=[18 19 20] (len=3)
Split  5 | train=[0 1 2]...[18 19 20] (len=21) | test=[21 22 23] (len=3)

Recipe B: sliding window, horizon=2, gap=3, max_train_size=12
Split  1 | train=[3 4 5]...[12 13 14] (len=12) | test=[18 19] (len=2)
Split  2 | train=[5 6 7]...[14 15 16] (len=12) | test=[20 21] (len=2)
Split  3 | train=[7 8 9]...[16 17 18] (len=12) | test=[22 23] (len=2)
Split  4 | train=[ 9 10 11]...[18 19 20] (len=12) | test=[24 25] (len=2)
Split  5 | train=[11 12 13]...[20 21 22] (len=12) | test=[26 27] (len=2)
Split  6 | train=[13 14 15]...[22 23 24] (len=12) | test=[28 29] (len=2)

Recipe C: many micro‑tests (test_size=1)
Split  1 | train=[0 1 2]...[5 6 7] (len=8) | test=[8] (len=1)
Split

## 5) Mini Project: Walk‑Forward Evaluation with a Simple Model
We build lag features and evaluate with `TimeSeriesSplit`.

In [11]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import TimeSeriesSplit

# Synthetic series with trend + noise
rng = np.random.default_rng(42)
T = 80
y = pd.Series(0.3*np.arange(T) + rng.normal(scale=1.0, size=T))

# Build supervised dataset: predict y_t from [y_{t-1}, y_{t-2}, y_{t-3}]
LAGS = 3
X = pd.concat({f"lag{i}": y.shift(i) for i in range(1, LAGS+1)}, axis=1).dropna()
y_super = y.loc[X.index]

# CV with leakage guard: gap=LAGS
tscv = TimeSeriesSplit(n_splits=5, test_size=5, gap=LAGS)
maes = []
for i, (tr, te) in enumerate(tscv.split(X), 1):
    model = LinearRegression()
    model.fit(X.iloc[tr], y_super.iloc[tr])
    pred = model.predict(X.iloc[te])
    mae = mean_absolute_error(y_super.iloc[te], pred)
    maes.append(mae)
    print(f"Split {i}: MAE={mae:.3f} | train_end_idx={tr[-1]} | test_idx={list(te)}")

print(f"\nMean MAE across splits: {np.mean(maes):.3f} ± {np.std(maes):.3f}")

Split 1: MAE=0.999 | train_end_idx=48 | test_idx=[np.int64(52), np.int64(53), np.int64(54), np.int64(55), np.int64(56)]
Split 2: MAE=1.108 | train_end_idx=53 | test_idx=[np.int64(57), np.int64(58), np.int64(59), np.int64(60), np.int64(61)]
Split 3: MAE=0.621 | train_end_idx=58 | test_idx=[np.int64(62), np.int64(63), np.int64(64), np.int64(65), np.int64(66)]
Split 4: MAE=0.874 | train_end_idx=63 | test_idx=[np.int64(67), np.int64(68), np.int64(69), np.int64(70), np.int64(71)]
Split 5: MAE=0.581 | train_end_idx=68 | test_idx=[np.int64(72), np.int64(73), np.int64(74), np.int64(75), np.int64(76)]

Mean MAE across splits: 0.837 ± 0.206


## 6) Parameter Selection Guidelines (Engineering View)
- **`test_size`** = your **forecast horizon** (e.g., next 1/7/30 steps).
- **`gap`** ≥ **max feature look‑back** (e.g., if you compute 14‑step rolling stats, use `gap≥14`).
- **`max_train_size`**: Use small values for **concept drift**; `None` when older data **still helps**.
- **`n_splits`**: Start 3–5; balance more evaluation points vs sufficient train length.

## 7) Common Pitfalls / จุดพลาดที่พบบ่อย
1. **Shuffling** time series before split → leakage.
2. `gap` too small for your feature lags → subtle leakage.
3. `n_splits` too large for dataset length → empty/invalid folds.
4. Misaligned features/targets after shifting → row mismatch.
5. Comparing results to random CV scores → unfair.

## 8) Exercises / แบบฝึกหัด
1. **Tune the horizon**: For `T=60`, set `n_splits=4`. Try `test_size=1`, `3`, `7`. How do MAE/SMAPE change?
2. **Leakage check**: If you use lags up to 5, experiment with `gap=0` vs `gap=5`. Observe score inflation with `gap=0`.
3. **Sliding vs Expanding**: Compare `max_train_size=None` vs `max_train_size=24` on a non‑stationary series.
4. **Edge cases**: With a 12‑point series, find the largest `n_splits` that still yields valid folds when `test_size=2`.

## 9) Back to Your Minimal Example (complete)

In [12]:
import numpy as np
from sklearn.model_selection import TimeSeriesSplit

X = np.array([[1, 2], [3, 4], [1, 2], [3, 4], [1, 2], [3, 4]])
y = np.array([1, 2, 3, 4, 5, 6])

print("Example 1: n_splits=3 (auto test_size)")
tscv = TimeSeriesSplit(n_splits=3)
for i, (train_index, test_index) in enumerate(tscv.split(X)):
    print(f"Split {i+1} -> TRAIN: {train_index}, TEST: {test_index}")

print("\nExample 2: n_splits=2, test_size=2")
tscv = TimeSeriesSplit(n_splits=2, test_size=2)
for i, (train_index, test_index) in enumerate(tscv.split(X)):
    print(f"Split {i+1} -> TRAIN: {train_index}, TEST: {test_index}")

print("\nExample 3: n_splits=2, test_size=1, gap=1")
tscv = TimeSeriesSplit(n_splits=2, test_size=1, gap=1)
for i, (train_index, test_index) in enumerate(tscv.split(X)):
    print(f"Split {i+1} -> TRAIN: {train_index}, TEST: {test_index}")

Example 1: n_splits=3 (auto test_size)
Split 1 -> TRAIN: [0 1 2], TEST: [3]
Split 2 -> TRAIN: [0 1 2 3], TEST: [4]
Split 3 -> TRAIN: [0 1 2 3 4], TEST: [5]

Example 2: n_splits=2, test_size=2
Split 1 -> TRAIN: [0 1], TEST: [2 3]
Split 2 -> TRAIN: [0 1 2 3], TEST: [4 5]

Example 3: n_splits=2, test_size=1, gap=1
Split 1 -> TRAIN: [0 1 2], TEST: [4]
Split 2 -> TRAIN: [0 1 2 3], TEST: [5]


## 10) Thai Summary (สรุปภาษาไทย)
- ใช้ `TimeSeriesSplit` สำหรับข้อมูลอนุกรมเวลาเพื่อหลีกเลี่ยงการรั่วไหลของอนาคต (data leakage).
- ตั้ง `test_size` ให้ตรงกับขอบเขตการพยากรณ์ (horizon).
- หากมีคุณลักษณะแบบ rolling/lag สูงสุดที่ `L` ให้ตั้ง `gap ≥ L`.
- ใช้ `max_train_size` เพื่อทำ **sliding window** เมื่อข้อมูลมี drift; ปล่อย `None` เมื่อข้อมูลคงที่กว่า.
- ปรับ `n_splits` เพื่อสมดุลจำนวนรอบประเมินกับขนาดข้อมูลใน train.

> Tip: เริ่มด้วย `n_splits=3–5`, `test_size` = horizon, `gap` = max lag, แล้วค่อยปรับตามผลลัพธ์